

---


###**Kunal Kwatra**
###**19BCE1784**
### **ML_PROJECT**

---






---


# **TITLE : INTRUSION DETECTION USING CVAE**

---





---


# **INFERENCE**

The above project depends on Conditional Variational Autoencoder, and aims at detection of intrusion in the network flow by classifying a record as one of the type of intrusion or normal by analyzing it's attributes.
For the first data is processed using variety of techniques including, removing 6 of the attributes, one-hot encoding, normalizing using min-max scalar, conversion of class labels and SMOTE to handle the imbalanced data.
Following that data is feeded to the self implemented CVAE model.
In the implementation of CVAE model, the model is divided into 2 parts - Encoder and Decoder.

For Encoder, the model takes 116 features of training data as input, passing it through 2 hidden layers with 500 neurons and 'relu' activation function. Followed to that we extract 2 layers of 25 neurons each for mean and log(sigma) of the data.

For the mean and log(sigma) data extracted from the encoder, sampling is performed to form a input of shape (None,25) for the decoder consisting a set of both mean and log(sigma) of the data.
For Decoder, model takes the latent variables(production from sampling of output of encoder) along with the 5 unique labels assigned to the iput data.
The first layer of decoder with 495 neurons and relu activation function, takes latent variables as the input. The output from this layer along with 5 unique labels is passed as input for the 2nd hidden layer with 500 neurons and relu activation function.
Finally in out[ut is produced by passing it throut the last layer providing the predicted 116 features.

For the whole process loss is calculated, in two parts - KL_loss and reconstruction loss, and is used to train the model.

For the trained model, data with 116 features is feeded and can be used for both classification of the class label for the given input and reconstrtuction of data.

For the implementation above, my contribution is implementation of CVAE right from defining of all the layers, calculation of loss to processing of the data in the form of shape and type to be fed to the model for training and testing.
The implemented model is fully functional and can be used for both label prediction of network record and feature reconstruction.

---



In [1]:
#mounting drive to import dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Libraries Imported**

In [2]:
#importing libraries
# for making dataframe and basic processing actions
import pandas as pd 
#importing tensorflow as object tf
import tensorflow as tf 
#Importing neural netwoek library - keras
from tensorflow import keras
#importing types of layes for the model
from tensorflow.keras.layers import Input,Dropout,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
%matplotlib inline

## Training and Testing Dataframe

In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/ML_Project/Dataset/nsl-kdd/KDDTrain+.txt')
df_train.drop('20', axis='columns', inplace=True)
df_train #Training dataframe

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,2,2.1,0.00,0.00.1,0.00.2,0.00.3,1.00,0.00.4,0.00.5,150,25,0.17.1,0.03,0.17.2,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,udp,private,SF,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [4]:
df_test = pd.read_csv("/content/drive/MyDrive/ML_Project/Dataset/nsl-kdd/KDDTest+.txt")
df_test.drop('21', axis='columns', inplace=True)
df_test #testing dataframe

,0,tcp,private,REJ,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,229,10,0.00,0.00.1,1.00,1.00.1,0.04,0.06,0.00.2,255,10.1,0.04.1,0.06.1,0.00.3,0.00.4,0.00.5,0.00.6,1.00.2,1.00.3,neptune
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0,tcp,http,SF,267,14515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.00,0.0,0.0,1.00,0.00,0.00,155,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.00,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.00,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.00,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


In [5]:
df_train.columns = ['duration',	'protocol_type',	'service',	'flag',	'src_bytes',	'dst_bytes',	'land',	'wrong_fragment',	'urgent',	'hot',	'num_failed_logins',	
                    'logged_in',	'num_compromised',	'root_shell',	'su_attempted',	'num_root',	'num_file_creations',	'num_shells',	'num_access_files',	'num_outbound_cmds',	
                    'is_host_login',	'is_guest_login',	'count',	'srv_count',	'serror_rate',	'srv_serror_rate',	'rerror_rate',	'srv_rerror_rate',	'same_srv_rate',	
                    'diff_srv_rate',	'srv_diff_host_rate',	'dst_host_count',	'dst_host_srv_count',	'dst_host_same_srv_rate',	'dst_host_diff_srv_rate',	'dst_host_same_src_port_rate',	
                    'dst_host_srv_diff_host_rate',	'dst_host_serror_rate',	'dst_host_srv_serror_rate',	'dst_host_rerror_rate',	'dst_host_srv_rerror_rate',	'class']
df_train                    

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,udp,private,SF,105,145,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [6]:
df_test.columns = ['duration',	'protocol_type',	'service',	'flag',	'src_bytes',	'dst_bytes',	'land',	'wrong_fragment',	'urgent',	'hot',	'num_failed_logins',	
                    'logged_in',	'num_compromised',	'root_shell',	'su_attempted',	'num_root',	'num_file_creations',	'num_shells',	'num_access_files',	'num_outbound_cmds',	
                    'is_host_login',	'is_guest_login',	'count',	'srv_count',	'serror_rate',	'srv_serror_rate',	'rerror_rate',	'srv_rerror_rate',	'same_srv_rate',	
                    'diff_srv_rate',	'srv_diff_host_rate',	'dst_host_count',	'dst_host_srv_count',	'dst_host_same_srv_rate',	'dst_host_diff_srv_rate',	'dst_host_same_src_port_rate',	
                    'dst_host_srv_diff_host_rate',	'dst_host_serror_rate',	'dst_host_srv_serror_rate',	'dst_host_rerror_rate',	'dst_host_srv_rerror_rate',	'class']
df_test    #Assigning column names 

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0,tcp,http,SF,267,14515,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.00,0.0,0.0,1.00,0.00,0.00,155,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.00,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.00,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.00,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


## Removing 6 features with majority values as 0's and 1's

In [7]:
#Checking which of the attributes have maximum values as 0s, to be removed
for col in df_train.columns:
  if(df_train[col].nunique() <5):
    print("\n"+col+" : Unique values : " + str(df_train[col].nunique()) + " : " + str(df_train[col].unique()))


protocol_type : Unique values : 3 : ['udp' 'tcp' 'icmp']

land : Unique values : 2 : [0 1]

wrong_fragment : Unique values : 3 : [0 3 1]

urgent : Unique values : 4 : [0 1 3 2]

logged_in : Unique values : 2 : [0 1]

root_shell : Unique values : 2 : [0 1]

su_attempted : Unique values : 3 : [0 1 2]

num_shells : Unique values : 3 : [0 1 2]

num_outbound_cmds : Unique values : 1 : [0]

is_host_login : Unique values : 2 : [0 1]

is_guest_login : Unique values : 2 : [0 1]


In [8]:
for col in df_test.columns:
  if(df_test[col].nunique() <5):
    print("\n"+col+" : Unique values : " + str(df_test[col].nunique()) + " : " + str(df_test[col].unique()))


protocol_type : Unique values : 3 : ['tcp' 'icmp' 'udp']

land : Unique values : 2 : [0 1]

wrong_fragment : Unique values : 3 : [0 1 3]

urgent : Unique values : 4 : [0 1 2 3]

logged_in : Unique values : 2 : [0 1]

root_shell : Unique values : 2 : [0 1]

su_attempted : Unique values : 3 : [0 2 1]

num_shells : Unique values : 4 : [0 1 2 5]

num_outbound_cmds : Unique values : 1 : [0]

is_host_login : Unique values : 2 : [0 1]

is_guest_login : Unique values : 2 : [0 1]


In [9]:
df_train.drop(['land', 'logged_in', 'root_shell', 'num_outbound_cmds', 'is_host_login', 'is_guest_login'], axis = 'columns', inplace=True)
df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,tcp,http,SF,232,8153,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,184,25,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,udp,private,SF,105,145,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,0,0,0,0,0,0,144,8,1.0,1.0,0.0,0.0,0.06,0.05,0.00,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [10]:
df_test.drop(['land', 'logged_in', 'root_shell', 'num_outbound_cmds', 'is_host_login', 'is_guest_login'], axis = 'columns', inplace=True)
df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0,tcp,http,SF,267,14515,0,0,0,0,0,0,0,0,0,0,4,4,0.0,0.00,0.0,0.0,1.00,0.00,0.00,155,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0,tcp,http,SF,317,938,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.00,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,54540,8314,0,0,2,0,1,0,0,0,0,0,5,10,0.0,0.00,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.00,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


## Normalizing

In [11]:
#checking for the datatypes of features of dataframe
df_train.dtypes

duration                         int64
protocol_type                   object
service                         object
flag                            object
src_bytes                        int64
dst_bytes                        int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
num_compromised                  int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_rate                float64
rerror_rate                    float64
srv_rerror_rate                float64
same_srv_rate                  float64
diff_srv_rate                  float64
srv_diff_host_rate             float64
dst_host_count           

In [12]:
#Using Min-Max Feature : series = (series-min(series))/(max(series)-min(series))
for col in df_train.columns:
  if col != 'protocol_type' and col != 'service' and col != 'flag' and col != 'class':
    df_train[col] = (df_train[col] - df_train[col].min())/(df_train[col].max() - df_train[col].min())


df_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.000000,udp,other,SF,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025440,0.001957,0.0,0.0,0.0,0.0,0.08,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0.000000,tcp,private,S0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240705,0.011742,1.0,1.0,0.0,0.0,0.05,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0.000000,tcp,http,SF,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009785,0.009785,0.2,0.2,0.0,0.0,1.00,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0.000000,tcp,http,SF,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058708,0.062622,0.0,0.0,0.0,0.0,1.00,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0.000000,tcp,private,REJ,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.236791,0.037182,0.0,0.0,1.0,1.0,0.16,0.06,0.00,1.000000,0.074510,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0.000000,tcp,private,S0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360078,0.048924,1.0,1.0,0.0,0.0,0.14,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,0.000186,udp,private,SF,7.608895e-08,1.106923e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0.000000,tcp,smtp,SF,1.616709e-06,2.931438e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0.000000,tcp,klogin,S0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281800,0.015656,1.0,1.0,0.0,0.0,0.06,0.05,0.00,1.000000,0.031373,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [13]:
#Using Min-Max Feature : series = (series-min(series))/(max(series)-min(series))
for col in df_test.columns:
  if col != 'protocol_type' and col != 'service' and col != 'flag' and col != 'class':
    df_test[col] = (df_test[col] - df_test[col].min())/(df_test[col].max() - df_test[col].min())

df_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.000000,tcp,private,REJ,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.266145,0.001957,0.0,0.00,1.0,1.0,0.01,0.06,0.00,1.000000,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,0.000035,tcp,ftp_data,SF,2.066513e-04,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.525490,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0.000000,icmp,eco_i,SF,3.183413e-07,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.127202,0.0,0.00,0.0,0.0,1.00,0.00,1.00,0.011765,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,0.000017,tcp,telnet,RSTO,0.000000e+00,0.000011,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.015656,0.0,0.12,1.0,0.5,1.00,0.00,0.75,0.113725,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0.000000,tcp,http,SF,4.249857e-06,0.010784,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007828,0.007828,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.607843,1.000000,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0.000000,tcp,smtp,SF,1.263815e-05,0.000247,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.392157,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0.000000,tcp,http,SF,5.045710e-06,0.000697,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.003914,0.021526,0.0,0.00,0.0,0.0,1.00,0.00,0.18,0.772549,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0.000000,tcp,http,SF,8.681168e-04,0.006177,0.0,0.0,0.019802,0.0,0.001256,0.0,0.0,0.0,0.0,0.0,0.009785,0.019569,0.0,0.00,0.0,0.0,1.00,0.00,0.20,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0.000000,udp,domain_u,SF,6.685168e-07,0.000031,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007828,0.011742,0.0,0.00,0.0,0.0,1.00,0.00,0.33,1.000000,0.988235,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


## One-Hot Encoding for features with catagorical values 

In [14]:
#for training data
#defining alias for the converted catagorical features
temp1 = pd.get_dummies(df_train.protocol_type, prefix='protocol_type')
temp2 = pd.get_dummies(df_train.service, prefix='service')
temp3 = pd.get_dummies(df_train.flag, prefix='flag')
#Removing the catagroiacal features from dataframe
df_train.drop(['protocol_type', 'flag', 'service'], axis='columns', inplace=True)

In [15]:
#Inserting the alias of converted catagorical features to the dataframe
i=1
for col in temp1:
  df_train.insert(i, col, temp1[col])
  i = i+1
for col in temp2:
  df_train.insert(i, col, temp2[col])
  i = i+1
for col in temp3:
  df_train.insert(i, col, temp3[col])
  i = i+1
df_train

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_harvest,service_hostnames,service_http,service_http_2784,service_http_443,service_http_8001,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025440,0.001957,0.0,0.0,0.0,0.0,0.08,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240705,0.011742,1.0,1.0,0.0,0.0,0.05,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009785,0.009785,0.2,0.2,0.0,0.0,1.00,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058708,0.062622,0.0,0.0,0.0,0.0,1.00,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.236791,0.037182,0.0,0.0,1.0,1.0,0.16,0.06,0.00,1.000000,0.074510,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360078,0.048924,1.0,1.0,0.0,0.0,0.14,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,0.000186,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,7.608895e-08,1.106923e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.616709e-06,2.931438e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [16]:
#for testing data
#defining alias for the converted catagorical features
temp1 = pd.get_dummies(df_test.protocol_type, prefix='protocol_type')
temp2 = pd.get_dummies(df_test.service, prefix='service')
temp3 = pd.get_dummies(df_test.flag, prefix='flag')
#Removing the catagroiacal features from dataframe
df_test.drop(['protocol_type', 'flag', 'service'], axis='columns', inplace=True)

In [17]:
#Inserting the alias of converted catagorical features to the dataframe
i=1
for col in temp1:
  df_test.insert(i, col, temp1[col])
  i = i+1
for col in temp2:
  df_test.insert(i, col, temp2[col])
  i = i+1
for col in temp3:
  df_test.insert(i, col, temp3[col])
  i = i+1
df_test

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,service_netbios_dgm,service_netbios_ns,service_netbios_ssn,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.266145,0.001957,0.0,0.00,1.0,1.0,0.01,0.06,0.00,1.000000,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,0.000035,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2.066513e-04,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.525490,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3.183413e-07,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.127202,0.0,0.00,0.0,0.0,1.00,0.00,1.00,0.011765,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,0.000017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000011,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.015656,0.0,0.12,1.0,0.5,1.00,0.00,0.75,0.113725,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,4.249857e-06,0.010784,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007828,0.007828,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.607843,1.000000,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.263815e-05,0.000247,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.392157,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,5.045710e-06,0.000697,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.003914,0.021526,0.0,0.00,0.0,0.0,1.00,0.00,0.18,0.772549,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,8.681168e-04,0.006177,0.0,0.0,0.019802,0.0,0.001256,0.0,0.0,0.0,0.0,0.0,0.009785,0.019569,0.0,0.00,0.0,0.0,1.00,0.00,0.20,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,6.685168e-07,0.000

# Changing Labels

In [18]:
#Types of attacks
dos_attacks=["snmpgetattack","back","land","neptune","smurf","teardrop","pod","apache2","udpstorm","processtable","mailbomb"]
r2l_attacks=["snmpguess","worm","httptunnel","named","xlock","xsnoop","sendmail","ftp_write","guess_passwd","imap","multihop","phf","spy","warezclient","warezmaster"]
u2r_attacks=["sqlattack","buffer_overflow","loadmodule","perl","rootkit","xterm","ps"]
probe_attacks=["ipsweep","nmap","portsweep","satan","saint","mscan"]

In [19]:
# Our new labels
classes = {"Normal": 0, "Dos": 1, "R2L": 2, "U2R" : 3, "Probe": 4}
#Funtion to allot the new label for a record passed as attribute
def label_attack (l):
    if l in dos_attacks:
        return classes["Dos"]
    if l in r2l_attacks:
        return classes["R2L"]
    if l in u2r_attacks:
        return classes["U2R"]
    if l in probe_attacks:
        return classes["Probe"]
    return classes["Normal"]

In [20]:
#For trainging data
c=[]
for l in df_train['class']:
  c.append(label_attack(l))
#Changing labels to new labels = 0,1,2,3,4
df_train.drop("class", axis=1, inplace=True)
df_train['label'] = c
df_train

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_harvest,service_hostnames,service_http,service_http_2784,service_http_443,service_http_8001,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025440,0.001957,0.0,0.0,0.0,0.0,0.08,0.15,0.00,1.000000,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
1,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240705,0.011742,1.0,1.0,0.0,0.0,0.05,0.07,0.00,1.000000,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
2,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009785,0.009785,0.2,0.2,0.0,0.0,1.00,0.00,0.00,0.117647,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
3,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058708,0.062622,0.0,0.0,0.0,0.0,1.00,0.00,0.09,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
4,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.236791,0.037182,0.0,0.0,1.0,1.0,0.16,0.06,0.00,1.000000,0.074510,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.360078,0.048924,1.0,1.0,0.0,0.0,0.14,0.06,0.00,1.000000,0.098039,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,1
125968,0.000186,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,7.608895e-08,1.106923e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003914,0.003914,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.956863,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0
125969,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.616709e-06,2.931438e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.0,0.0,0.0,1.00,0.00,0.00,1.000000,0.117647,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,0
125970,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281800,0.015656,1.0,1.0,0.0,0.0,0.06,0.

In [21]:
#For testing data
c=[]
for l in df_test['class']:
  c.append(label_attack(l))
#Changing labels to new labels = 0,1,2,3,4
df_test.drop("class", axis=1, inplace=True)
df_test['label'] = c
df_test

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,service_netbios_dgm,service_netbios_ns,service_netbios_ssn,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.266145,0.001957,0.0,0.00,1.0,1.0,0.01,0.06,0.00,1.000000,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,1
1,0.000035,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2.066513e-04,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.525490,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,0
2,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,3.183413e-07,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.127202,0.0,0.00,0.0,0.0,1.00,0.00,1.00,0.011765,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,4
3,0.000017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000e+00,0.000011,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.015656,0.0,0.12,1.0,0.5,1.00,0.00,0.75,0.113725,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,4
4,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,4.249857e-06,0.010784,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007828,0.007828,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.607843,1.000000,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1.263815e-05,0.000247,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.392157,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,0
22539,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,5.045710e-06,0.000697,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.003914,0.021526,0.0,0.00,0.0,0.0,1.00,0.00,0.18,0.772549,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,0
22540,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,8.681168e-04,0.006177,0.0,0.0,0.019802,0.0,0.001256,0.0,0.0,0.0,0.0,0.0,0.009785,0.019569,0.0,0.00,0.0,0.0,1.00,0.00,0.20,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,1
22541,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,6.685168e-07,0.000031,0.0,0.0,0.000000,0.0,0.000000,0.0

In [22]:
#new labels of training data
df_train["label"].unique()

array([0, 1, 2, 4, 3])

In [23]:
#new labels of testing data
df_test["label"].unique()

array([1, 0, 4, 2, 3])

# Testing and Training data

In [24]:
train_x = df_train.iloc[:, 0:116] #Features of training data
train_y = df_train["label"] # Labels for training data

## **CVAE Model Training**

In [25]:
from tensorflow.keras import layers

#orignal shape of trainginfg data is (125972, 116)
original_dim = 116 
# Number of nureons in internal layers
intermediate_dim = 500 
 # Special internal layer dimension for a particular layer of decoder to be comined with the labels associated with the inputs for decoder after sampling
id2 = 495
#Dimension of latent variables (output of encoder) 
latent_dim = 25 
#Dimension of labels passed to decoder 
l_dim = 5 

# Encoder-Decoder

In [26]:
 #Taking 116 features of training data as input for encoder
inputs = keras.Input(shape=(original_dim,))
#Taking 5 labels as input for decoder
label_inputs = keras.Input(shape=(5,), name='label_5') 


#ENCODER
#First hidden layer with 500 neurons and using relu activation funtion taking input, the 116 features of traing dataset
he1 = layers.Dense(intermediate_dim, activation='relu')(inputs) 
#Second hidden layer with 500 neurons and using relu activation funtion taking the output of first hidden layer as input
he2 = layers.Dense(intermediate_dim, activation='relu')(he1)
# Layer with 25 neurons for the mean taking output from second layer as input 
z_mean = layers.Dense(latent_dim)(he2) 
# Layer with 25 neurons for sigma taking output from second layer as input
z_log_sigma = layers.Dense(latent_dim)(he2) 


#Sampling
from keras import backend as K

#Sampling for latent variables to be passed as input for decoder
def sampling(args):
    z_mean, z_log_sigma = args
    sigma = 0.5*z_log_sigma
    epsilon = K.random_normal(shape=(K.shape(sigma)[0], latent_dim))
    return z_mean + K.exp(sigma) * epsilon

#Special layer lamda to call the defined function sampling
z = layers.Lambda(sampling)([z_mean, z_log_sigma])


#DECODER
#First hidden layer with 495 neurons and using relu activation funtion taking input z
hd1 = layers.Dense(id2, activation='relu')(z)
#Second hidden layer with 500 neurons and using relu activation funtion taking the output of first hidden layer concatenated with the 5 unique labels as input
hd2 = layers.Dense(intermediate_dim, input_dim=2, activation="relu")(keras.layers.Concatenate(axis=1)([hd1, label_inputs]))
#output layer of decoder taking input as the output of layer hd2 and using a non linear activation funtion - sigmoid activation function
outputs = layers.Dense(original_dim, activation='sigmoid')(hd2)


#CVAE Model
cvae = keras.Model([inputs, label_inputs], outputs, name='ae_mlp')


#Loss Calculation
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
#Normalised total caluculated loss of the model
cvae_loss = K.mean(reconstruction_loss + kl_loss)
cvae.add_loss(cvae_loss)


#Compiling the formed cvae model using the adam optimizer
cvae.compile(optimizer='adam')

In [27]:
#Sum up of the whole cvae model formed
cvae.summary()

Model: "ae_mlp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 116)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 500)          58500       ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 500)          250500      ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 25)           12525       ['dense_1[0][0]']                
                                                                                             

# Input_tensors

In [28]:
#5 unique labels to be passed as input
l = pd.get_dummies(train_y, prefix='label')

In [29]:
l

,label_0,label_1,label_2,label_3,label_4
0,1,0,0,0,0
1,0,1,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
125967,0,1,0,0,0
125968,1,0,0,0,0
125969,1,0,0,0,0
125970,0,1,0,0,0


In [30]:
#Forming tensor of the unique labels
l_tensor = tf.convert_to_tensor(l, dtype=tf.float32)

In [31]:
l_tensor

<tf.Tensor: shape=(125972, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)>

In [32]:
#Forming tensor of the training data features to passed as input
x_t = tf.convert_to_tensor(train_x, dtype=tf.float32) 

# Fitting of CVAE Model

In [33]:
#Fitting the formed CVAE model over the traing dataset
history = cvae.fit([x_t, l_tensor], x_t, batch_size = 256, epochs = 100, shuffle = True, validation_split = 0.20);

Epoch 1/100
394/394 [==============================] - 12s 29ms/step - loss: 11.3068 - val_loss: 8.0107
Epoch 2/100
394/394 [==============================] - 11s 28ms/step - loss: 7.5036 - val_loss: 7.1672
Epoch 3/100
394/394 [==============================] - 11s 28ms/step - loss: 7.0030 - val_loss: 6.8496
Epoch 4/100
394/394 [==============================] - 11s 28ms/step - loss: 6.7676 - val_loss: 6.6621
Epoch 5/100
394/394 [==============================] - 11s 28ms/step - loss: 6.6301 - val_loss: 6.5876
Epoch 6/100
394/394 [==============================] - 11s 28ms/step - loss: 6.5269 - val_loss: 6.4800
Epoch 7/100
394/394 [==============================] - 11s 28ms/step - loss: 6.4553 - val_loss: 6.4465
Epoch 8/100
394/394 [==============================] - 11s 29ms/step - loss: 6.3985 - val_loss: 6.3682
Epoch 9/100
394/394 [==============================] - 11s 28ms/step - loss: 6.3520 - val_loss: 6.3437
Epoch 10/100
394/394 [==============================] - 11s 28ms/step - 

# Testing 

In [34]:
#Testing the fitted model with training data
pred_train = cvae.predict([x_t, l_tensor])
pred_train

array([[9.0723425e-02, 3.2174587e-04, 2.4574101e-03, ..., 5.8762409e-05,
        3.9654970e-04, 7.5980738e-06],
       [3.0413879e-08, 3.4619002e-11, 1.0000000e+00, ..., 9.9999660e-01,
        1.2764335e-04, 1.8894910e-06],
       [7.1356408e-06, 6.0657194e-06, 9.9997956e-01, ..., 1.4069378e-03,
        4.4291914e-03, 8.5372925e-03],
       ...,
       [1.4129281e-04, 2.3692136e-08, 9.9999976e-01, ..., 2.3932576e-02,
        9.0160549e-02, 1.8518865e-03],
       [5.4236984e-09, 6.3224821e-16, 1.0000000e+00, ..., 9.9996686e-01,
        2.8534860e-05, 1.1763375e-06],
       [3.4421810e-05, 1.6713825e-07, 9.9998093e-01, ..., 8.0818832e-03,
        1.9742340e-02, 8.0919266e-04]], dtype=float32)

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
import numpy
#Formig a numpy array having it's values as 0
zarray = numpy.zeros(22543)

In [37]:
#As the testing dataframe has 6 less features, so adding 6 columns with value 0 to have a least effect on the classification
for i in range(6):
  df_test["dummy"+str(i+1)] = zarray

In [38]:
df_test

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_auth,service_bgp,service_courier,service_csnet_ns,service_ctf,service_daytime,service_discard,service_domain,service_domain_u,service_echo,service_eco_i,service_ecr_i,service_efs,service_exec,service_finger,service_ftp,service_ftp_data,service_gopher,service_hostnames,service_http,service_http_443,service_imap4,service_iso_tsap,service_klogin,service_kshell,service_ldap,service_link,service_login,service_mtp,service_name,service_netbios_dgm,service_netbios_ns,service_netbios_ssn,...,flag_SF,flag_SH,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,su_attempted,num_root,num_file_creations,num_shells,num_access_files,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,dummy1,dummy2,dummy3,dummy4,dummy5,dummy6
0,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000e+00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.266145,0.001957,0.0,0.00,1.0,1.0,0.01,0.06,0.00,1.000000,0.003922,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,1,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000035,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2.066513e-04,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.525490,0.337255,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3.183413e-07,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.127202,0.0,0.00,0.0,0.0,1.00,0.00,1.00,0.011765,0.223529,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,4,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000e+00,0.000011,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.015656,0.0,0.12,1.0,0.5,1.00,0.00,0.75,0.113725,0.337255,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,4,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,4.249857e-06,0.010784,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007828,0.007828,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.607843,1.000000,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,1.263815e-05,0.000247,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001957,0.001957,0.0,0.00,0.0,0.0,1.00,0.00,0.00,0.392157,0.552941,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0
22539,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,5.045710e-06,0.000697,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.003914,0.021526,0.0,0.00,0.0,0.0,1.00,0.00,0.18,0.772549,1.000000,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0
22540,0.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,8.681168e-04,0.006177,0.0,0.0,0.019802,0.0,0.001256,0.0,0.0,0.0,0.0,0.0,0.009785,0.019569,0.0,0.00,0.0,0.0,1.00,0.00,0.20,1.000000,1.000000,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,1,0.0,0.0,0.0,0.0,0.0,0.0
22541,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [39]:
#5 unique labels of testing data
l_test = pd.get_dummies(df_test["label"], prefix='label')
l_test

,label_0,label_1,label_2,label_3,label_4
0,0,1,0,0,0
1,1,0,0,0,0
2,0,0,0,0,1
3,0,0,0,0,1
4,1,0,0,0,0
...,...,...,...,...,...
22538,1,0,0,0,0
22539,1,0,0,0,0
22540,0,1,0,0,0
22541,1,0,0,0,0


In [40]:
#Forming tensor for features of testing data
x_tensor = tf.convert_to_tensor(df_test.iloc[:, 0:116], dtype=tf.float32)
#Foriming tensor for labels of test data
label_tensor = tf.convert_to_tensor(l_test)
#applying trained cvae model over the testing data 
pred_test = cvae.predict([x_tensor, label_tensor])
pred_test

array([[8.1363204e-08, 1.9945886e-14, 1.0000000e+00, ..., 9.9999750e-01,
        9.8803239e-06, 7.6154294e-07],
       [1.1377633e-03, 1.4103299e-09, 1.0000000e+00, ..., 2.7670562e-02,
        1.9487232e-02, 2.8696954e-03],
       [3.5636683e-07, 4.6730930e-10, 8.6610949e-01, ..., 9.0470910e-04,
        7.5999373e-01, 7.7078141e-07],
       ...,
       [2.8330636e-08, 1.7014119e-14, 1.0000000e+00, ..., 9.9999893e-01,
        2.6735568e-06, 4.5811213e-08],
       [4.1912139e-05, 5.0373842e-06, 9.9996889e-01, ..., 3.1243265e-03,
        3.8785338e-03, 9.2291832e-03],
       [1.2673140e-03, 1.4365809e-05, 9.0844780e-01, ..., 3.4678984e-01,
        2.7647644e-01, 6.7056066e-01]], dtype=float32)

# **INFERENCE**

The above project depends on Conditional Variational Autoencoder, and aims at detection of intrusion in the network flow by classifying a record as one of the type of intrusion or normal by analyzing it's attributes.
For the first data is processed using variety of techniques including, removing 6 of the attributes, one-hot encoding, normalizing using min-max scalar, conversion of class labels and SMOTE to handle the imbalanced data.
Following that data is feeded to the self implemented CVAE model.
In the implementation of CVAE model, the model is divided into 2 parts - Encoder and Decoder.

For Encoder, the model takes 116 features of training data as input, passing it through 2 hidden layers with 500 neurons and 'relu' activation function. Followed to that we extract 2 layers of 25 neurons each for mean and log(sigma) of the data.

For the mean and log(sigma) data extracted from the encoder, sampling is performed to form a input of shape (None,25) for the decoder consisting a set of both mean and log(sigma) of the data.
For Decoder, model takes the latent variables(production from sampling of output of encoder) along with the 5 unique labels assigned to the iput data.
The first layer of decoder with 495 neurons and relu activation function, takes latent variables as the input. The output from this layer along with 5 unique labels is passed as input for the 2nd hidden layer with 500 neurons and relu activation function.
Finally in out[ut is produced by passing it throut the last layer providing the predicted 116 features.

For the whole process loss is calculated, in two parts - KL_loss and reconstruction loss, and is used to train the model.

For the trained model, data with 116 features is feeded and can be used for both classification of the class label for the given input and reconstrtuction of data.

For the implementation above, my contribution is implementation of CVAE right from defining of all the layers, calculation of loss to processing of the data in the form of shape and type to be fed to the model for training and testing.
The implemented model is fully functional and can be used for both label prediction of network record and feature reconstruction.